# Retrieval Augmented Generation (RAG) with Langchain em Português!
*Using IBM Granite Models*

## In this notebook
This notebook contains instructions for performing Retrieval Augumented Generation (RAG). RAG is an architectural pattern that can be used to augment the performance of language models by recalling factual information from a knowledge base, and adding that information to the model query. The most common approach in RAG is to create dense vector representations of the knowledge base in order to retrieve text chunks that are semantically similar to a given user query.

RAG use cases include:
- Customer service: Answering questions about a product or service using facts from the product documentation.
- Domain knowledge: Exploring a specialized domain (e.g., finance) using facts from papers or articles in the knowledge base.
- News chat: Chatting about current events by calling up relevant recent news articles.

In its simplest form, RAG requires 3 steps:

- Initial setup:
  - Index knowledge-base passages for efficient retrieval. In this recipe, we take embeddings of the passages and store them in a vector database.
- Upon each user query:
  - Retrieve relevant passages from the database. In this recipe, we use an embedding of the query to retrieve semantically similar passages.
  - Generate a response by feeding retrieved passage into a large language model, along with the user query.

## Setting up the environment

### Python version

Ensure you are running Python 3.10 or 3.11.

In [1]:
import sys
assert sys.version_info >= (3, 10) and sys.version_info < (3, 12), "Use Python 3.10 or 3.11 to run this notebook."

### Install dependencies

Granite Kitchen comes with a bundle of dependencies that are required for notebooks. See the list of packages in its [`setup.py`](https://github.com/ibm-granite-community/granite-kitchen/blob/main/setup.py). 

In [2]:
%%capture

%pip install "git+https://github.com/ibm-granite-community/granite-kitchen.git" "langchain-huggingface" "langchain-milvus" "wget"

### Serving the Granite AI model


This notebook requires IBM Granite models to be served by an AI model runtime so that the models can be invoked or called. This notebook can use a locally accessible [Ollama](https://github.com/ollama/ollama) server to serve the models, or the [Replicate](https://replicate.com) cloud service.

During the pre-work, you may have either started a local Ollama server on your computer, or setup Replicate access and obtained an [API token](https://replicate.com/account/api-tokens).

## Selecting System Components

### Choose your Embeddings Model

Specify the model to use for generating embedding vectors from text.

To use a model from a provider other than Huggingface, replace this code cell with one from [this Embeddings Model recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Embeddings_Models.ipynb).

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="ibm-granite/granite-embedding-278m-multilingual")

/Users/flaviajaninebeo/Documents/Virtualenvs/granite-workshop/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Choose your Vector Database

Specify the database to use for storing and retrieving embedding vectors.

To connect to a vector database other than Milvus substitute this code cell with one from [this Vector Store recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Vector_Stores.ipynb).

In [4]:
from langchain_milvus import Milvus
import tempfile

db_file = tempfile.NamedTemporaryFile(prefix="milvus_br_", suffix=".db", delete=False).name
print(f"The vector database will be saved to {db_file}")

vector_db = Milvus(embedding_function=embeddings_model, connection_args={"uri": db_file}, auto_id=True)

The vector database will be saved to /var/folders/5x/cztshy892cbf92p2fdgqlxhc0000gn/T/milvus_br_owitv3ea.db


## Select your model


Select a Granite model to use. Here we use a Langchain client to connect to the model. If there is a locally accessible Ollama server, we use an Ollama client to access the model. Otherwise, we use a Replicate client to access the model.

When using Replicate, if the `REPLICATE_API_TOKEN` environment variable is not set, or a `REPLICATE_API_TOKEN` Colab secret is not set, then the notebook will ask for your [Replicate API token](https://replicate.com/account/api-tokens) in a dialog box.

In [6]:
import os
import requests
from langchain_ollama.llms import OllamaLLM
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import set_env_var

try: # Look for a locally accessible Ollama server for the model
    response = requests.get(os.getenv("OLLAMA_HOST", "http://127.0.0.1:11434"))
    model = OllamaLLM(model="granite3.2:8b")
except Exception: # Use Replicate for the model
    set_env_var("REPLICATE_API_TOKEN")
    model = Replicate(model="ibm-granite/granite-3.0-8b-instruct")


In [7]:
model.invoke(input="hello")

'Hello! How can I assist you today?'

## Building the Vector Database

In this example, we take the State of the Union speech text, split it into chunks, derive embedding vectors using the embedding model, and load it into the vector database for querying.

### Download the document

Here we use President Biden's State of the Union address from March 1, 2022.

In [9]:
import wget

filename = 'crepioca.pdf'
url = 'https://light4you.com.br/cardapio/itens-individuais1/salgados/crepioca-de-frango'

if not os.path.isfile(filename):
  wget.download(url, out=filename)

### Split the document into chunks

Split the document into text segments that can fit into the model's context window.

In [7]:
%%capture
%pip install --upgrade unstructured

I0000 00:00:1744049098.151039 10380393 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [8]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter

loader = UnstructuredURLLoader([ "https://light4you.com.br/cardapio/itens-individuais1/salgados/crepioca-de-frango" ])
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

### Populate the vector database

NOTE: Population of the vector database may take over a minute depending on your embedding model and service.

In [9]:
vector_db.add_documents(texts)

[457192008700133376,
 457192008700133377,
 457192008700133378,
 457192008700133379,
 457192008700133380,
 457192008700133381,
 457192008700133382,
 457192008700133383,
 457192008700133384,
 457192008700133385,
 457192008700133386,
 457192008700133387,
 457192008700133388]

## Querying the Vector Database

### Conduct a similarity search

Search the database for similar documents by proximity of the embedded vector in vector space.

In [10]:
query = "O que é crepioca?"
docs = vector_db.similarity_search(query)
print(docs[0].page_content)

Voltar

Crepioca_Frango

Crepioca de Frango com Requeijão Light

Cód. Barras: 0123000

138 calorias

Uma opção deliciosa para você comer no meio da tarde ou também serve de pré ou pós treino ou mesmo de jantar com uma salada!

R$ 14,50

Comprar

Aguarde

Calcular frete

Calcular

aquarde


## Answering Questions

### Automate the RAG pipeline

Build a RAG chain with the model and the document retriever. See the [Granite Prompting Guide](https://www.ibm.com/granite/docs/models/granite/#prompt-anatomy) for information on the prompt template.

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create a prompt template for question-answering with the retrieved context.
prompt_template = """\
<|start_of_role|>user<|end_of_role|>Use os seguintes pedaços de contexto para responder à pergunta no final.
Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta. 
Responda sempre em Português.

{context}

Pergunta: {input}<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>"""

# Assemble the retrieval-augmented generation chain.
qa_chain_prompt = PromptTemplate.from_template(prompt_template)
combine_docs_chain = create_stuff_documents_chain(model, qa_chain_prompt)
rag_chain = create_retrieval_chain(vector_db.as_retriever(), combine_docs_chain)

### Generate a retrieval-augmented response to a question

Use the RAG chain to process the query. 

In [12]:
output = rag_chain.invoke({"input": "Como fazer uma crepioca?"})

print(output['answer'])

Para fazer uma crepioca, você precisa de alguns ingredientes principais: farinha de trigo, ovos, leite, sal e pimenta. Você também precisa de um refratário ou um prato fritura adequado para o air fryer.

Primeiro, pré-aqueça o air fryer para uma temperatura de 350°F (175°C). Em seguida, preparar a massa da crepioca. Em uma tigela grande, bata os ovos e adiciona o leite, sal e pimenta. Méxua até ficar suave e bem combinado.

Em outra tigela, misture a farinha de trigo com um pouco de água para formar uma massa muito macia. Adicione a massa macia à mistura dos ovos e leite, mescando até obter uma massa homogênea.

Agora, é tempo para cozinhar a crepioca. Despeje a massa na panela do air fryer e coza por cerca de 10 minutos ou até que fique dourada. Deixe esfriar um pouco a crepioca antes de servir com sua preferida requeijão light ou salada.

Bom apetite!


In [13]:
model.invoke(qa_chain_prompt.format(context=None, input="Como fazer uma crepioca?"))

'Desculpe, mas não tenho informações sobre como fazer uma crepioca. Crepiocas são alimentos tradicionais de Portugal, mas a forma como eles são feitos pode variar. Se você puder fornecer mais detalhes ou fontes, posso tentar ajudar melhor.'

In [14]:
for i in range(10):
    print("RAG:", rag_chain.invoke({"input": "Como fazer uma crepioca?"})['answer']) 
    print("Puro:", model.invoke(qa_chain_prompt.format(context=None, input="Como fazer uma crepioca?")))

RAG: Para fazer uma crepioca, você precisa ter os seguintes ingredientes:

* Farinha de trigo
* Azeite de oliva
* Sal
* Pepa
* Ovo
* Queijo parmesano (opcional)

Aqui está um recipoe simples para fazer uma crepioca:

1. Preparar a massa: Em uma tigela, mesclar a farinha de trigo com o azeite de oliva, a sal e a pepa. Adicione o ovo e méite até obter uma massa homogênea. Se preferir, adicione o queijo parmesano para dar um sabor mais saboroso.
2. Formar as crepiocas: Preparar uma superfície ligeiramente enchada com farinha de trigo. Tire uma pequena quantidade da massa e esfregue a mão para formar uma bola. Coloque-a em uma planta de cozinha lavada e repita até formar todas as crepiocas necessárias.
3. Aquecer as crepiocas: Em um panela grande, coloque alguns centímetros de azeite de oliva e aqueça ao alto. Quando estiver quente, coloque uma ou duas crepiocas na panela e aqueça por cerca de 2-3 minutos, ou até que estejam douradas. Use um colher para retirá-las da panela e colocá-las em

In [16]:
rag_chain.invoke({"input": "Como fazer uma crepioca?"})

{'input': 'Como fazer uma crepioca?',
 'context': [Document(metadata={'pk': 457192008700133376, 'source': 'https://light4you.com.br/cardapio/itens-individuais1/salgados/crepioca-de-frango'}, page_content='Voltar\n\nCrepioca_Frango\n\nCrepioca de Frango com Requeijão Light\n\nCód. Barras: 0123000\n\n138 calorias\n\nUma opção deliciosa para você comer no meio da tarde ou também serve de pré ou pós treino ou mesmo de jantar com uma salada!\n\nR$ 14,50\n\nComprar\n\nAguarde\n\nCalcular frete\n\nCalcular\n\naquarde'),
  Document(metadata={'pk': 457192008700133385, 'source': 'https://light4you.com.br/cardapio/itens-individuais1/salgados/crepioca-de-frango'}, page_content='6. Bom apetite :)\n\nAQUECIMENTO EM AIR FRYER\n\n1. Abre a embalagem e coloque todos os produtos que desejar consumir congelados na air fryer.\n\n2. Ligue e deixe agir por cerca de 9 minutos.\n\n3. Coloque no prato com cuidado para não se queimar.'),
  Document(metadata={'pk': 457192008700133382, 'source': 'https://light4yo